# VAEGAN NEGATIVE BINOMIAL

> The VAEGAN model, takes an Variational Encoder, Variational Decoder and Classifier model as inputs. Uses a negative binomial as the latent variable rather than Gausian.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| default_exp Models.VAEGAN_NEG_BI

In [ ]:
#| export
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import scvi
from scvi.models.distributions import NegativeBinomial
from torch.distributions import Normal

class VAEGAN_NEG_BI(nn.Module):
    def __init__(self, encoder_z, encoder_l, decoder_x, decoder_r, decoder_p, classifier, log = True):
        """
        The VAEGAN model with Negative Binomial distribution as Latent Variable
        """
        super(VAEGAN_NEG_BI, self).__init__()
        self.encoder_z = encoder_z
        self.encoder_l = encoder_l
        self.decoder_x = decoder_x
        self.decoder_r = decoder_r
        self.decoder_p = decoder_p
        
        self.classifier = classifier
    
        
    def reparameterize(self, mu, logvar):
        var = torch.exp(logvar) + 1e-4
        z = Normal(mu, var.sqrt()).rsample()
        return z
    
    def decode(self, z, l, train = True):
        h_x = self.decoder_x(z) 
        h_x = F.relu(h_x)
        h_r = self.decoder_r(h_x)      
        h_r = F.softmax(h_r)
        if train :
            h_p = torch.exp(torch.clamp(l, max=12)) * h_r
        else :
            h_p = torch.exp(l) * h_r
        x_hat = NegativeBinomial(mu = h_r, theta= h_p).sample()
        return x_hat, h_r, h_p


    def forward(self, x, log=True):
        if log :
            x= torch.log(x+1)
        mu_l, logvar_l = self.encoder_l(x)
        mu_z, logvar_z = self.encoder_z(x)
        
        l = self.reparameterize(mu_l, logvar_l)
        z = self.reparameterize(mu_z, logvar_z)
        
        x_hat, h_r, h_p = self.decode(z,l)
        
        y_hat = self.classifier(z) 
        
        return x_hat, y_hat, mu_z, logvar_z, h_r, h_p, l



Global seed set to 0
/Users/eamonmcandrew/opt/anaconda3/envs/scINTEGRATION/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/eamonmcandrew/opt/anaconda3/envs/scINTEGRATION/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
def NLL_loss(data, h_r, h_p):
    
    data = data.view(-1, 1)
    ll = torch.distributions.negative_binomial.NegativeBinomial(h_r, h_p)

    neg_ll = -torch.mean(torch.sum(ll, dim=-1))
    return neg_ll